In [26]:
client = MongoClient(host = "mongodb://localhost:27017/")
    
db = client.db_agg
transactions = db.transactions

In [35]:
# Q-1. Calculer le montant total des paiements ?
# Indice : key : Payment.Total
pipeline = [
    {"$unwind": "$Panier"},
    {"$group": {"_id": None, "total": {"$sum": "$Panier.price" }}},
]
        
r1 = [ t for t in transactions.aggregate(pipeline) ]
print("Question 1:\n{}".format(r1))

Question 2:
[{'_id': None, 'total': 1640}]


In [30]:
# Q-2. Calculer le montant total par transactions (montant total du panier) ?
pipeline = [
    {"$unwind": "$Panier"},
    {"$group": {"_id": "$TransactionId", "total": {"$sum": "$Panier.price" }}},
]
        
r2 = [ t for t in transactions.aggregate(pipeline) ]
print("Question 2:\n{}".format(r2))

Question 2:
[{'_id': 'tran4', 'total': 410}, {'_id': 'tran3', 'total': 410}, {'_id': 'tran2', 'total': 410}, {'_id': 'tran1', 'total': 410}]


In [31]:
# Q-3. Calculate total payments (Payment.Total) for each payment type (Payment.Type) ?

matching = {"$match":{"Payment":{"$ne": None}}}
TotalPayment = {"$group" : {"_id" : "$Payment.Type", "Total Payment" : { "$sum" : "$Payment.Total" }}}
for t in transactions.aggregate( [matching,TotalPayment] ):
    print(t)

{'_id': 'Debit-Card', 'Total Payment': 410}
{'_id': 'Credit-Card', 'Total Payment': 820}


In [36]:
# Q-4. Trouver l'Id le plus élevé.
group = {"$group": { "_id" : None , "Id": {"$max": "$Id"} }}
sort = {"$sort" : {"max" : -1}}

for t in transactions.aggregate([group, sort]):
    print(t)

{'_id': None, 'Id': 103}


In [33]:
# Q-5. Find the max price (Transaction.price) ?
eclater = { "$unwind" : '$Panier' }
maxPrice = { "$group" : {"_id" : None , "max_price" : { "$max" : "$Panier.price" }} }
tri = {"$sort" : {"max" : -1}}
for t in transactions.aggregate( [eclater, maxPrice, tri] ):
    print(t)

{'_id': None, 'max_price': 210}
